![IFMG](https://images.even3.com.br/sBWZnWPFUBgLOGciSZc4G5ZQy7Q=/1100x440/smart/even3.blob.core.windows.net/banner/ARTEPARASITE3.993b3db9f908426e9833.png)

---

# Introdução ao Tensorflow - Parte 3

---

#### Professor: Felipe Reis

#### Data: 20-10-2021

---
### Importação de bibliotecas 

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import random
import time

#import plot_images
from xject_helper import xject_helper as helper

#tensorflow
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras

---
## Base de Dados CIFAR-10

A base de dados CIFAR-10 consiste em 60.000 imagens coloridas de 32x32, divididas em 10 classes.
Cada classe contém 6.000 imagens por classe. 
O conjunto é dividido em 50.000 imagens de treinamento e 10.000 imagens de teste.

CIFAR é o acrônimo de Canadian Institute For Advanced Research, proprietário da base de dados.
O CIFAR-10 e CIFAR-100 são subconjuntos rotulados de uma base de dados de pequenas imagens que contém 80 milhões de amostras. 

As imagens foram coletadas por Alex Krizhevsky, Vinod Nair e Geoffrey Hinton.

A base está disponível em: https://www.cs.toronto.edu/~kriz/cifar.html

Ela também está disponível na página do Tensorflow: https://www.tensorflow.org/datasets/catalog/cifar10

![CIFAR 10](https://cdn-images-1.medium.com/max/1200/1*SZnidBt7CQ4Xqcag6rd8Ew.png)

Fonte: *Jannik Zürn. **Training a CIFAR-10 classifier in the cloud using TensorFlow and Google Colab**. 2018. Disponível em: https://jannik-zuern.medium.com/training-a-cifar-10-classifier-in-the-cloud-using-tensorflow-and-google-colab-f3a5fbdfe24d. Acesso em: 05 de outubro de 2021.*

In [ ]:
#download da base de dados
cifar10 = keras.datasets.cifar10

#divisão em base de treinamento e testes (subdividido em imagens e labels)
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

#remove 1 dimensão dos arrays de label
train_labels = np.squeeze(train_labels)
test_labels = np.squeeze(test_labels)

In [ ]:
#impressão da quantidade de imagens da base
print('Conjunto treino: {} amostras'.format(len(train_images)))
print('Conjunto teste: {} amostras'.format(len(test_images)))

In [ ]:
#definicao dos nomes das classes
class_names = ['Avião', 'Automóvel', 'Pássaro', 'Gato', 'Veado', 
               'Cachorro', 'Sapo', 'Cavalo', 'Barco', 'Caminhão']

### Análise dos Dados

In [ ]:
helper.plot_images(train_images, train_labels, class_names, random=False)

---
## Pré processamento

Podemos classificar o conjunto de dados utilizando os mesmos modelos utilizados no último no Notebook 1.

In [ ]:
#pré processamento
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
helper.plot_images(train_images, train_labels, class_names, random=False)

---
## Treinamento do Modelo

O modelo de rede neural abaixo é baseado na arquitetura existente no Notebook 1.

Para adequação ao conjunto CIFAR-10, a entrada foi alterada para suportar imagens de tamanho 32 $\times$ 32 pixels, com 3 canais de cores.

In [ ]:
def load_model():
    #modelo de rede sequencial
    return keras.Sequential([
        #transforma a image em um array de imagens de três dimensões (32 x 32 x 3)
        keras.layers.Flatten(input_shape=(32, 32, 3)), 

        #Camada de dados totalmente conectadas, com ativação relu
        #A camada possui 128 nós (neurônios)
        keras.layers.Dense(128, activation='relu'),

        #Camada de dados totalmente conectadas, com ativação softmax
        #A camada possui 10 nós (neurônios), correspondentes às probabilidades de cada classe (10 classes)
        keras.layers.Dense(10, activation='softmax')
    ])

---
## Compilação do Modelo


### Configurações de Compilação

Nesta seção serão detalhadas as algumas configurações de compilação do modelo, possibilitando alterar o otimizador, a função de perda e as métricas de qualidade.

Neste processo, concentraremos nosso estudo no comando `model.compile`.

Informações mais detalhadas e outros parâmetros podem ser encontradas no link: https://keras.io/api/models/model_training_apis/

---
### Funções de Perda

O objetivo da função de perda (*loss function*) é calcular o quanto o resultado predito pelo modelo é similar ao resultado esperado.

Nesse contexto, a função de perda busca minimizar a diferença entre os valores preditos e esperados. Em otimização, a função de perda (ou função de custo ou função de erro) é definido como o oposto da função objetivo.

A função de perda é uma função matemática cujo resultado não está limitado a um intervalo de valores. Com isso, a função de perda pode exibir resultados cuja compreensão pode não ser trivial (ex.: loss = 751.35). Esses valores são calculados e não são normalizados, ou seja, dependem da quantidade de amostras do conjunto, da quantidade de classes, do tamanho das imagens, entre outros atributos.

A comparação entre perdas de dois problemas diferentes pode não ser trivial.

Para alterar a função de perda, devemos utilizar o atributo `loss`.

Mais informações podem ser encontradas no link: https://keras.io/api/losses/

#### Funções de Perda para Classificação

Para treinamento da rede, podemos utilizar diferentes funções de perda. De acordo com a tarefa e a representação de dados utilizada, podemos escolher a função mais adequada. Seguem abaixo algumas funções de perda comuns.

A função `binary_crossentropy` calcula o [*cross entropy*](https://en.wikipedia.org/wiki/Cross_entropy) (entropia cruzada) entre rótulos verdadeiros e rótulos previstos, em problemas de classificação binária (0 ou 1). 

A função `categorical_crossentropy` calcula o [*cross entropy*](https://en.wikipedia.org/wiki/Cross_entropy) (entropia cruzada) entre rótulos verdadeiros e rótulos previstos em problemas de classificação categórica. Essa função, no entanto, requer que os rótulos sejam fornecidos em uma representação [*one_hot*](https://minerandodados.com.br/one-hot-encoding-como-funciona-python/). Esse tipo de codificação é usada para evitar que exista influência da codificação numérica das classes sobre o resultado final do algoritmo.

A função `sparse_categorical_crossentropy` calcula o [*cross entropy*](https://en.wikipedia.org/wiki/Cross_entropy) (entropia cruzada) entre rótulos verdadeiros e rótulos previstos em problemas de classificação categórica. Essa função é uma alternativa à função `categorical_crossentropy`, não exigindo que os dados estejam em formato [*one_hot*](https://minerandodados.com.br/one-hot-encoding-como-funciona-python/).

A função `kl_divergence` calcula a [divergência de Kullback-Leibler](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence) entre rótulos verdadeiros e rótulos previstos em problemas de classificação categórica.

A função `categorical_hinge` calcula o [*hinge loss*](https://en.wikipedia.org/wiki/Hinge_loss) entre rótulos verdadeiros e rótulos previstos em problemas de classificação categórica.  Essa função, no entanto, requer que os rótulos sejam fornecidos em uma representação [*one_hot*](https://minerandodados.com.br/one-hot-encoding-como-funciona-python/).

O Keras conta com outras funções de perda. Além disso, é possível programar outras funções de perda, para melhor desempenho do algoritmo (ou para tarefas cuja função de perda deva ter um comportamento distinto).


#### Funções de Perda para Regressão

Para problemas de regressão, podemos utilizar funções de perda mais adequadas à tarefa. 

Entre as principais funções de perda, destacam-se as seguintes funções:

- Similaridade por Cosseno ([Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity)), definida pelo comando `cosine_similarity`;
- Erro Quadrático Médio ([MSE - Mean Squared Error](https://en.wikipedia.org/wiki/Mean_squared_error)), definida pelo comando `mean_squared_error`;
- Erro Absoluto Médio ([MAE - Mean Absolute Error](https://en.wikipedia.org/wiki/Mean_absolute_error)) , definida pelo comando `mean_absolute_error`.

In [ ]:
#gera modelo
model1 = load_model()

#compila o modelo
model1.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
model1.fit(train_images, train_labels, epochs=10)

In [ ]:
#gera modelo
model1 = load_model()

#compila o modelo
model1.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

#treina o modelo
model1.fit(train_images, train_labels, epochs=10)

##### Função Categorical Cross Entropy

Podemos converter nossos rótulos (*labels*) para o formato *one hot*, com auxílio do código abaixo.

Essa transformação permite que utilizemos a função `categorical_crossentropy`.

In [ ]:
indices = train_labels
depth = 10

one_hot_train_labels = tf.one_hot(indices, depth)

print(train_images.shape)
print(one_hot_train_labels.shape)
print(one_hot_train_labels)

In [ ]:
#gera modelo
model1 = load_model()

#compila o modelo
model1.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
model1.fit(train_images, one_hot_train_labels, epochs=10)

---
### Métricas

A métrica é uma função usada para avaliar o desempenho de um modelo de rede neural.

As métricas não são utilizadas para treinamento da rede. Somente fornecem uma informação estatística a respeito da qualidade dos resultados (intermediários ou finais).

Ao contrário da função de perda, a métrica normalmente é limitada a um intervalo de valores, possibilitando a comparação de resultados de forma trivial. Devido a sua natureza, os valores, em geral, são limitados ao intervalo 0-100. 

Métricas não são adequadas como funções de perda uma vez que os resultados somente indicam "acertos" e "erros", sem expressar a evolução do treinamento (ex.: a métrica não melhorou, porém a previsão foi mais próxima do rótulo verdadeiro, indicando que o algoritmo está melhorando - o que é possível com a função de perda).

Em alguns cenários, pode ser vantajoso utilizar as próprias funções de perda (como em problemas de regressão). Não há restrições quanto a isso.

Para alterar, adicionar ou remover métricas, podemos o atributo `metrics`.

Mais informações podem ser encontradas no link: https://keras.io/api/metrics/


#### Métricas para Classificação

- [Acurácia](https://keras.io/api/metrics/accuracy_metrics/#accuracy-class);
- [Acurácia Binária](https://keras.io/api/metrics/accuracy_metrics/#binaryaccuracy-class);
- [Acurácia Categórica](https://keras.io/api/metrics/accuracy_metrics/#categoricalaccuracy-class);
- [Precisão](https://keras.io/api/metrics/classification_metrics/#precision-class);
- [Revocação](https://keras.io/api/metrics/classification_metrics/#recall-class);
- [AUC](https://keras.io/api/metrics/classification_metrics/#auc-class).

#### Métricas para Regressão

- [Cosine Similarity](https://keras.io/api/metrics/regression_metrics/#cosinesimilarity-class);
- [MSE](https://keras.io/api/metrics/regression_metrics/#meansquarederror-class);
- [MAE](https://keras.io/api/metrics/regression_metrics/#meanabsoluteerror-class).

#### Métricas para Segmentação

Interseção sobre União Média (*Mean Intersection-Over-Union*) é uma métrica de avaliação de segmentação semântica de imagens. Primeiro calcula o IOU para cada classe semântica e, em seguida, calcula a média sobre as classes.

- [MeanIOU](https://keras.io/api/metrics/segmentation_metrics/#meaniou-class).

In [ ]:
#gera modelo
model1 = load_model()

#compila o modelo
model1.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',
               metrics=['accuracy', 'binary_accuracy', tf.keras.metrics.SparseCategoricalAccuracy()])

#treina o modelo
model1.fit(train_images, train_labels, epochs=10)

#### Métricas Precisão e Revocação

Podemos utilizar as métricas Precisão e Revocação caso façamos a alteração da função de perda para `categorical_crossentropy`.

Caso contrário, o sistema acusa erro.

In [ ]:
#transforma o modelo em one hot
one_hot_train_labels = tf.one_hot(train_labels, 10)

#gera modelo
model1 = load_model()

#compila o modelo
model1.compile(optimizer='sgd', loss='categorical_crossentropy', 
               metrics=[tf.keras.metrics.Precision(name='precision'), 
                        tf.keras.metrics.Recall(name='recall'),
                        tf.keras.metrics.PrecisionAtRecall(0.5)])

#treina o modelo
model1.fit(train_images, one_hot_train_labels, epochs=10)

---
### Otimizadores

Otimizadores correspondem aos métodos numéricos utilizados para minimização do erro esperado.

O objetivo é reduzir a diferença entre a saída predita e a saída esperada.

No Tensorflow, o otimizador é definido pelo atributo `optimizer`, ao executar o método `compile` do modelo.

Podemos, por padrão usar os seguintes otimizadores:

- [SGD](https://keras.io/api/optimizers/sgd/);
- [RMSprop](https://keras.io/api/optimizers/rmsprop/);
- [Adam](https://keras.io/api/optimizers/adam/);
- [Adadelta](https://keras.io/api/optimizers/adadelta/);
- [Adagrad](https://keras.io/api/optimizers/adagrad/);
- [Adamax](https://keras.io/api/optimizers/adamax/);
- [Nadam](https://keras.io/api/optimizers/Nadam/);
- [Ftrl](https://keras.io/api/optimizers/ftrl/).

É possível configurar esses otimizadores, conforme veremos na sequência, de modo a customizá-los.

O Tensorflow ainda permite a criação de otimizadores próprios, que podem ser usados na tentativa de aumentar o desempenho da rede.

Mais informações podem ser encontradas no link: https://keras.io/api/optimizers/

In [ ]:
#gera modelo
model1 = load_model()

#compila o modelo
model1.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
model1.fit(train_images, train_labels, epochs=10)

In [ ]:
#gera modelo
model1 = load_model() #o modelo foi recarregado para melhor comparação dos otimizadores

#compila o modelo
model1.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
model1.fit(train_images, train_labels, epochs=10)

In [ ]:
#gera modelo
model1 = load_model() #o modelo foi recarregado para melhor comparação dos otimizadores

#compila o modelo
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
model1.fit(train_images, train_labels, epochs=10)

#### Taxa de Aprendizado (*Learning Rate*)

A taxa de aprendizado corresponde a taxa de atualização de pesos de uma rede neural. Pode ser entendida como o tamanho do passo em uma dada iteração diminuição da perda da rede.

A taxa de aprendizado é frequentemente denotada pela letra $\eta$ (eta).

Podemos definir explicitamente a taxa de aprendizado da rede, com auxílio dos otimizadores. Para isso, devemos utilizar instanciar um objeto correspondente ao otimizador que queremos utilizar e, em seguida, definir a taxa de aprendizado.

O valor ideal da taxa de aprendizado pode variar de acordo com o otimizador.

In [ ]:
#gera modelo
model1 = load_model() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
sgd = tf.keras.optimizers.SGD(learning_rate=0.01)

#compila o modelo
model1.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(train_images, train_labels, epochs=10)

# plota o gráfico
helper.plot_history_training(history, metrics=['accuracy', 'val_accuracy'], legend=['Treinamento', 'Validação'])

In [ ]:
#gera modelo
model1 = load_model() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(train_images, train_labels, epochs=10)

# plota o gráfico
helper.plot_history_training(history, metrics=['accuracy', 'val_accuracy'], legend=['Treinamento', 'Validação'])

#### Decaimento da Taxa de Aprendizado (*Learning Rate Decay*)

Ao definir a taxa de aprendizado, podemos querer que nossa rede inicie com um taxa de aprendizado e, em seguida, a taxa descresça constantemente.

Nesse caso, busca-se uma atualização de peso mais brusca no início do treinamento e uma atualização mais suave após algumas épocas, com objetivo de refinamento.

Para isso, podemos utilizar uma função de decaimento, que fará com que a função de perda diminua ao longo do tempo.

In [ ]:
#gera modelo
model1 = load_model() #o modelo foi recarregado para melhor comparação dos otimizadores

#define o decaimento da taxa de aprendizado
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

#define o otimizador
adam = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

#compila o modelo
model1.compile(optimizer=adam, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#treina o modelo
history = model1.fit(train_images, train_labels, epochs=10)

# plota o gráfico
helper.plot_history_training(history, metrics=['accuracy', 'val_accuracy'], legend=['Treinamento', 'Validação'])

----
----

## Tarefas 

### Tarefa 1 (30 minutos)

Treine a rede desenvolvida na Tarefa 1 - Notebook 2 no conjunto de dados CIFAR-10.

Altere os parâmetros de compilação, como otimizadores, funções de perda e as métricas de avaliação.

#### Atividades Sugeridas

- Altere os otimizadores e veja a diferença na acurácia;
- Altere as taxas de aprendizado e decaimento da rede;
- Teste algumas funções de perda distintas;
- Altere as métricas de avaliação: caso queira, adicione múltiplas métricas.
- Plote gráficos para verificar a existência de overfitting.

In [ ]:
#insira seu modelo aqui (arquitetura de rede)

In [ ]:
#compile seu modelo aqui

In [ ]:
#treine seu modelo aqui

In [ ]:
#imprima o gráfico de treinamento aqui

In [ ]:
#avalie seu modelo aqui

----

### Tarefa 2 (30 minutos)

Treine a rede desenvolvida na tarefa anterior no conjunto de dados CIFAR-100.

Comece com o melhor resultado obtido na tarefa anterior e altere os parâmetros de compilação, como otimizadores, taxas de aprendizado e funções de perda.

Informações sobre a base de dados: https://www.cs.toronto.edu/~kriz/cifar.html

![CIFAR 100](https://web.stanford.edu/~hastie/CASI_files/DATA/cifar100.jpg)

Fonte: *Trevor Hastie. **CIFAR-100 image database**. Disponível em: https://web.stanford.edu/~hastie/CASI_files/DATA/cifar-100.html. Acesso em: 16 de outubro de 2021.*

#### Atividades Sugeridas

- Altere os otimizadores e veja a diferença na acurácia;
- Altere as taxas de aprendizado e decaimento da rede;
- Teste algumas funções de perda distintas;
- Altere as métricas de avaliação: caso queira, adicione múltiplas métricas.
- Plote gráficos para verificar a existência de overfitting.

In [ ]:
#download da base de dados
cifar100 = keras.datasets.cifar100

#divisão em base de treinamento e testes (subdividido em imagens e labels)
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()

#remove 1 dimensão dos arrays de label
train_labels = np.squeeze(train_labels)
test_labels = np.squeeze(test_labels)

In [ ]:
#definicao dos nomes das classes CIFAR100
class_names_100 = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 
    'bed', 'bee', 'beetle', 'bicycle', 'bottle', 
    'bowl', 'boy', 'bridge', 'bus', 'butterfly', 
    'camel', 'can', 'castle', 'caterpillar', 'cattle', 
    'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 
    'couch', 'crab', 'crocodile', 'cup', 'dinosaur', 
    'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 
    'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 
    'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 
    'lobster', 'man', 'maple_tree', 'motorcycle', 'mountain', 
    'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid', 
    'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 
    'plain', 'plate', 'poppy', 'porcupine', 'possum', 
    'rabbit', 'raccoon', 'ray', 'road', 'rocket', 
    'rose', 'sea', 'seal', 'shark', 'shrew', 
    'skunk', 'skyscraper', 'snail', 'snake', 'spider', 
    'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table',
    'tank', 'telephone', 'television', 'tiger', 'tractor', 
    'train', 'trout', 'tulip', 'turtle', 'wardrobe', 
    'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]

In [ ]:
#insira seu modelo aqui (arquitetura de rede)

In [ ]:
#compile seu modelo aqui

In [ ]:
#treine seu modelo aqui

In [ ]:
#imprima o gráfico de treinamento aqui

In [ ]:
#avalie seu modelo aqui